### Define Objects for grid, PV-DER, events, simulation environment, and results

In [ ]:
#Import PV-DER modules
from pvder.DER_components import SolarPV_DER,PV_module
from pvder.grid_components import Grid
from pvder.dynamic_simulation import GridSimulation
from pvder.simulation_events import SimulationEvents
from pvder.utilities import SimulationUtilities,SimulationResults
from pvder import utility_functions

#Import Python modules
import numpy as np
import scipy as sci
import matplotlib as matplotlib
print('Scipy version:',sci.__version__)
print('Numpy version:',np.__version__)
print('Matplotlib version:',matplotlib.__version__)
#To show interactive plot in the notebook cell
%matplotlib notebook    

### Create objects

In [ ]:
events1 = SimulationEvents()
grid1 = Grid(events=events1,unbalance_ratio_b=1.0,unbalance_ratio_c=1.0)
PV_DER1 = SolarPV_DER(grid_model=grid1,events=events1,Sinverter_rated = 50.0e3,standAlone = True,STEADY_STATE_INITIALIZATION=False)
sim1 = GridSimulation(grid_model=grid1,PV_model=PV_DER1,simulation_events = events1)
results1 = SimulationResults(simulation = sim1,PER_UNIT=True)

### Add or remove events during simulation

In [ ]:
#Solar insolation event
events1.add_solar_event(3,90,298.15)

#Load event
events1.add_load_event(5,5.0 + 1j*0.0)

#Voltage event
events1.add_grid_event(6.0,0.98,60.0)

#Frequency event
events1.add_grid_event(7,1.00,59.0)

#Remove events if needed
#events1.remove_solar_event(3)
#events1.remove_grid_event(5)
#events1.remove_grid_event(6)
#events1.remove_load_event(7)


### Show events

In [ ]:
events1.show_events()

### Specify run time flags, simulation time, and run simulation 

In [ ]:
%%time
PV_DER1.MPPT_ENABLE=False
PV_DER1.RAMP_ENABLE = False
PV_DER1.VOLT_VAR_ENABLE = False
PV_DER1.LVRT_ENABLE = False
PV_DER1.LFRT_ENABLE = False
PV_DER1.DO_EXTRA_CALCULATIONS = True
sim1.jacFlag = True
sim1.DEBUG_SIMULATION = False
sim1.DEBUG_VOLTAGES = True
sim1.DEBUG_CURRENTS = True
sim1.DEBUG_POWER = False
sim1.DEBUG_CONTROLLERS  = True
sim1.DEBUG_PLL = False
sim1.PER_UNIT = True
sim1.DEBUG_SOLVER  = True
sim1.tStop = 20.0
sim1.tInc = 0.001
sim1.run_simulation()
PV_DER1.validate_model()

### Plot results

In [ ]:
results1.PER_UNIT = False
results1.PLOT_TITLE = True
results1.font_size = 18
results1.plot_DER_simulation(plot_type='active_power_Ppv_Pac_PCC')#

In [ ]:
results1.plot_DER_simulation(plot_type='reactive_power')#_Q_PCC

In [ ]:
results1.PER_UNIT = False
results1.plot_DER_simulation(plot_type='voltage_LV') #voltage_Vpcclv #_LV

In [ ]:
results1.PER_UNIT = False
results1.plot_DER_simulation(plot_type='voltage_Vdc')

In [ ]:
results1.PER_UNIT = False
results1.plot_DER_simulation(plot_type='current')

In [ ]:
results1.plot_DER_simulation(plot_type='frequency')